In [ ]:
import numpy as np
import random

In [ ]:
#a = np.load('../Data/G13_millennium/V.npy')
#b = np.load('../Data/G13_millennium/V_fixed.npy')

In [ ]:
log_centralmvir = np.log10(np.load('../Data/G13_millennium/centralmvir.npy') + 1e-10) + 10
type_gal = np.load('../Data/G13_millennium/type.npy')
fofid = np.load('../Data/G13_millennium/fofid.npy')
#stellarmass = np.load('../Data/G13_millennium/stellarmass.npy')
#sfr = np.load('../Data/G13_millennium/sfr.npy')
x = np.load('../Data/G13_millennium/x.npy')
y = np.load('../Data/G13_millennium/y.npy')
z = np.load('../Data/G13_millennium/z.npy')
r_vector = np.array([x,y,z]).T
del x,y,z

In [ ]:
logm_min = 10.
logm_max = 15.
NBIN = 30
mass_index = ((log_centralmvir - logm_min)/(logm_max - logm_min) * NBIN).astype(int)

In [ ]:
mask = (mass_index >= 0) & (mass_index < NBIN)
mass_index = mass_index[mask]
type_gal = type_gal[mask]
fofid = fofid[mask]
r_vector = r_vector[mask]
#sfr = sfr[mask]
del mask

In [ ]:
%%time
N,j,k = 0,0,0
V = np.zeros((len(mass_index),3))
len_data = len(mass_index)

for i in range(NBIN):
    central_positions = np.where((type_gal == 0) & (mass_index == i))[0]
    num_halos = len(central_positions)
    shuffled_pos = np.array(random.sample(central_positions, num_halos))
    for j in range(num_halos):
        
        pos_i = shuffled_pos[j]
        if j == num_halos - 1: pos_f = shuffled_pos[0]
        else: pos_f = shuffled_pos[j+1]
        v_dir = r_vector[pos_f] - r_vector[pos_i] 
        cond = True
        
        while cond:
            V[pos_i + N] = v_dir 
            N += 1  # Recorre el halo
            if pos_i + N == len_data: break
            cond = (fofid[pos_i+N] == fofid[pos_i])
            k += 1 # Recorre todas las galaxias
            
        N = 0
    

In [ ]:
np.save('../Data/G13_millennium/V.npy', V)

In [ ]:
%%time
def isout (r,vr):
    if r + vr < 0:
        return 1
    elif r + vr > 500: 
        return -1
    else:
        return 0

Lbox = 500.    
for i in range(len_data):
    
    x,y,z = r_vector[i]
    dx,dy,dz  = V[i]

    r_vector[i][0] = x + dx + isout(x,dx)*Lbox
    r_vector[i][1] = y + dy + isout(y,dy)*Lbox
    r_vector[i][2] = z + dz + isout(z,dz)*Lbox
    
    

In [ ]:
np.save('../Data/G13_millennium/r.npy', r_vector)